In [1]:
import aisuite as ai
import click
import os
from dotenv import load_dotenv
load_dotenv()
from vllm import LLM, SamplingParams
import json

# Huggingface

In [2]:
llm = LLM(model="meta-llama/Llama-3.1-8B-Instruct", max_model_len=4096, trust_remote_code=True)
tokenizer = llm.get_tokenizer()

INFO 01-08 11:40:36 config.py:510] This model supports multiple tasks: {'reward', 'classify', 'embed', 'score', 'generate'}. Defaulting to 'generate'.
INFO 01-08 11:40:36 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='meta-llama/Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 01-08 11:40:42 model_runner.py:1099] Loading model weights took 14.9888 GB
INFO 01-08 11:40:43 worker.py:241] Memory profiling takes 0.77 seconds
INFO 01-08 11:40:43 worker.py:241] the current vLLM instance can use total_gpu_memory (23.55GiB) x gpu_memory_utilization (0.90) = 21.20GiB
INFO 01-08 11:40:43 worker.py:241] model weights take 14.99GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 1.20GiB; the rest of the memory reserved for KV Cache is 4.92GiB.
INFO 01-08 11:40:43 gpu_executor.py:76] # GPU blocks: 2519, # CPU blocks: 2048
INFO 01-08 11:40:43 gpu_executor.py:80] Maximum concurrency for 4096 tokens per request: 9.84x
INFO 01-08 11:40:44 model_runner.py:1415] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilizat

Capturing CUDA graph shapes: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:14<00:00,  2.36it/s]

INFO 01-08 11:40:59 model_runner.py:1535] Graph capturing finished in 15 secs, took 0.26 GiB
INFO 01-08 11:40:59 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 17.15 seconds


In [67]:
import json

tool_list = json.load(open("../temp_tool_desc.json", "r"))["nodes"]

for tool in tool_list:
    parameter_list = []
    for parameter in tool["parameters"]:
        parameter_list.append({
            "name": parameter["name"],
            "description": parameter["desc"]
        })
    tool["parameters"] = parameter_list

In [68]:
# Tool documentation 생성 함수
def generate_tool_document(tool_list):
    document = "TOOL DOCUMENTATION\n"
    document += "Below is a list of tools with their descriptions and required parameters.\n"

    for tool in tool_list:
        document += f"TOOL ID: {tool['function']}\n"
        document += f"DESCRIPTION: {tool['desc']}\n"
        document += "PARAMETERS:\n"
        for param in tool['parameters']:
            document += f"  - {param['name']} ({param.get('type', 'string')}): {param['description']}\n"
        document += "\n"  # 각 도구를 구분하기 위한 빈 줄 추가

    return document

# Tool documentation 생성
tool_document = generate_tool_document(tool_list)

# 결과 출력
print(tool_document)

TOOL DOCUMENTATION
Below is a list of tools with their descriptions and required parameters.
TOOL ID: get_weather
DESCRIPTION: Get the weather for a specific city and a specific day
PARAMETERS:
  - location (string): The location to get the weather for
  - date (string): The date to get the weather for. The date format should follow the followng: MM-DD.

TOOL ID: get_news_for_topic
DESCRIPTION: Get the news for a specific topic
PARAMETERS:
  - topic (string): The topic to get the news for

TOOL ID: stock_operation
DESCRIPTION: Do a specific operation on a specific stock
PARAMETERS:
  - stock (string): The stock to do the operation on
  - operation (string): The operation to do, eg. buy, sell, short, cover etc.

TOOL ID: book_flight
DESCRIPTION: Book a flight for a specific date, from a specific location to a specific destination
PARAMETERS:
  - date (string): The date to book the flight for. The date format should follow the followng: MM-DD.
  - from (string): The location to book the 

## With Reasoning

In [69]:
prompt = """# TOOL DOCUMENT #:
{{tool_document}}

# GOAL #: 
Using the tools provided in # TOOL DOCUMENT #, generate the required functions and their parameter values to fully address the users' request in # DIALOGUE #, especially the last turn of what the user said. 
Only produce output in the JSON format specified below. **Do not include any additional text, explanations, or content outside the JSON format.**

# OUTPUT FORMAT #: 
```
{
  "functions": [
    {
      "function_name": "name_of_the_function",
      "function_reasoning": "reason_for_selecting_the_function",
      "parameters": [
        {
          "parameter_name": "name_of_the_parameter",
          "parameter_value": "value_of_the_parameter",
          "parameter_type": "type_of_the_parameter (e.g., string, integer, boolean, etc.)",
          "parameter_reasoning": "reason_for_selecting_the_parameter"
        }
      ]
    }
  ]
}
```

# REQUIREMENTS #: 
1. Each function must directly address the users' request in # DIALOGUE # and align with tools in the # TOOL DOCUMENT #.
2. Provide the actual values for the parameters, and functions based on the context of the dialogue whenever possible.
3. Define each parameter with its name, value, type, and a brief explanation of its role directly under "parameter_reasoning".
4. **Only generate output in the JSON format specified in # OUTPUT FORMAT #. Do not include any explanations, additional comments, or additional text outside the JSON.**
5. If there are no functions or parameters required based on the dialogue, return an empty JSON object: `{ "functions": [] }`.

# DIALOGUE #: 
{{dialogue}}

Now generate your result using the specified format excluding any additional text, explanations, or content outside the JSON format:
# RESULT #:
"""

In [70]:
input_file_path = '../generate_dialogue/handmade_dataset/SM.json'  # 파일 경로
with open(input_file_path, 'r') as file:
    data = json.load(file)

# 멀티 파티 대화로 변환하는 함수
def transform_to_multi_party(data):
    transformed_conversations = []
    
    for conversation in data['conversations']:
        metadata = conversation['metadata']
        users = metadata['users']
        messages = conversation['messages']
        
        # 새로운 대화 형식 생성
        transformed = []
        
        # 시스템 메시지 추가
        # transformed.append({
        #     "role": "system",
        #     "content": f"You are an assistant specializing in {metadata['category'].lower()}."
        # })
        
        # 사용자 메시지 변환
        for message in messages:
            # 메시지 발화자의 이름과 메타데이터 찾기
            user_name = message['user']
            user_metadata = None
            # for user_id, user_info in users.items():
                # if user_info['persona_name'] == user_name:
                #     user_metadata = {"persona": user_info['persona']}
                #     break
            
            # 새로운 메시지 형식 추가
            transformed.append({
                "role": "user",
                "name": user_name,
                # "metadata": user_metadata,
                "content": message['message']
            })
        
        # 변환된 대화 저장
        transformed_conversations.append(transformed)
    
    return transformed_conversations

# 변환 실행
temp_dialogues = transform_to_multi_party(data)

import pprint
print(pprint.pprint(transformed_data[0]))

[{'content': "Broccoli is so healthy and versatile these days. Wouldn't it be "
             'easier to just buy it online?',
  'name': 'Taylor',
  'role': 'user'},
 {'content': "True, but where's the best place to get it? I usually shop at "
             'Walmart, and their produce is pretty fresh.',
  'name': 'Hyunjin',
  'role': 'user'},
 {'content': 'Do you think getting broccoli from Walmart is environmentally '
             'friendly? I’m curious if it’s organic or if the packaging is '
             'sustainable.',
  'name': 'Sujin',
  'role': 'user'},
 {'content': 'Well, I made pasta with broccoli from Walmart before, and it was '
             'fresh and tasty. They even had organic options.',
  'name': 'Taylor',
  'role': 'user'},
 {'content': 'Walmart’s prices are pretty reasonable, too. Plus, their '
             'delivery is fast, so you can get what you need right away.',
  'name': 'Hyunjin',
  'role': 'user'},
 {'content': 'But isn’t buying from Walmart less eco-friendly c

In [71]:
tokenizer.chat_template = """
{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- set date_string = "26 Jul 2024" %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{# System message 처리 #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content']|trim %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}
{{- "Cutting Knowledge Date: December 2023\n" }}
{{- "Today Date: " + date_string + "\n\n" }}
{{- system_message }}
{{- "<|eot_id|>" }}

{# Messages 처리 (name 포함) #}
{%- for message in messages %}
    {%- if message.name is defined %}
        {{- "<|start_header_id|>" + message.role + "<|end_header_id|>\n\n" + message.name + " (" + message.role + "): " + message.content|trim + "<|eot_id|>" }}
    {%- else %}
        {{- "<|start_header_id|>" + message.role + "<|end_header_id|>\n\n" + message.content|trim + "<|eot_id|>" }}
    {%- endif %}
{%- endfor %}

{# Assistant 응답 생성 지점 #}
{%- if add_generation_prompt %}
    {{- "<|start_header_id|>assistant<|end_header_id|>\n\n" }}
{%- endif %}
"""

In [72]:
def apply_template(dialogue):
    dialogue = tokenizer.apply_chat_template(
        dialogue,
        tokenize=False,
        add_generation_prompt=True
    )
    return dialogue

# temp_dialogue = tokenizer.apply_chat_template(
#     temp_dialogue,
#     tokenize=False,
#     add_generation_prompt=True
# )
# print(temp_dialogue)

In [73]:
# sampling_params = SamplingParams(temperature=0.85, top_p=0.95, max_tokens=1024, repetition_penalty=1.1, stop_token_ids=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")])
sampling_params = SamplingParams(temperature=0.85, top_p=0.95, max_tokens=1024, repetition_penalty=1.1)


outputs = []

for temp_dialogue in temp_dialogues:
    input_prompt = prompt.replace('{{tool_document}}', tool_document).replace('{{dialogue}}', apply_template(temp_dialogue))

    output = llm.generate(input_prompt, sampling_params)
    outputs.append(output)


Processed prompts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.36s/it, est. speed input: 1288.70 toks/s, output: 95.53 toks/s]

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.73s/it, est. speed input: 338.78 toks/s, output: 51.41 toks/s]

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.09s/it, est. speed input: 405.08 toks/s, output: 51.08 toks/s]

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.53s/it, est. speed input: 598.2

In [75]:
print(outputs[0][0].prompt)

# TOOL DOCUMENT #:
TOOL DOCUMENTATION
Below is a list of tools with their descriptions and required parameters.
TOOL ID: get_weather
DESCRIPTION: Get the weather for a specific city and a specific day
PARAMETERS:
  - location (string): The location to get the weather for
  - date (string): The date to get the weather for. The date format should follow the followng: MM-DD.

TOOL ID: get_news_for_topic
DESCRIPTION: Get the news for a specific topic
PARAMETERS:
  - topic (string): The topic to get the news for

TOOL ID: stock_operation
DESCRIPTION: Do a specific operation on a specific stock
PARAMETERS:
  - stock (string): The stock to do the operation on
  - operation (string): The operation to do, eg. buy, sell, short, cover etc.

TOOL ID: book_flight
DESCRIPTION: Book a flight for a specific date, from a specific location to a specific destination
PARAMETERS:
  - date (string): The date to book the flight for. The date format should follow the followng: MM-DD.
  - from (string): The lo

In [81]:
print(outputs[4][0].outputs[0].text)

```
{
  "functions": [
    {
      "function_name": "book_hotel",
      "function_reasoning": "to book a suitable hotel in Jeju considering the user's preferences",
      "parameters": [
        {
          "parameter_name": "date",
          "parameter_value": "03-15",
          "parameter_type": "string",
          "parameter_reasoning": "the date the user plans to stay in the hotel"
        },
        {
          "parameter_name": "location",
          "parameter_value": "Jeju",
          "parameter_type": "string",
          "parameter_reasoning": "the location where the user wants to book a hotel"
        }
      ]
    },
    {
      "function_name": "find_hotel",
      "function_reasoning": "to search for available hotels in Jeju with a beach view",
      "parameters": [
        {
          "parameter_name": "check_in",
          "parameter_value": "03-15",
          "parameter_type": "string",
          "parameter_reasoning": "the check-in date for the hotel search"
        },
 

In [10]:
with open('graph_desc.json', 'r') as file:
    data = json.load(file)
print(len(data))

2


In [11]:
len(data['nodes'])

40

In [12]:
len(data['links'])

1560